In [1]:
!pip install llmlingua
!pip install openai
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.7 MB/s e

In [23]:
import openai
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [24]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-",
)

In [4]:
!git clone https://github.com/FranxYao/chain-of-thought-hub.git

Cloning into 'chain-of-thought-hub'...
remote: Enumerating objects: 3043, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 3043 (delta 191), reused 247 (delta 179), pack-reused 2704
Receiving objects: 100% (3043/3043), 8.30 MiB | 26.81 MiB/s, done.
Resolving deltas: 100% (964/964), done.


In [5]:
!cd

In [6]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('./chain-of-thought-hub/gsm8k/lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [25]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [26]:
gsm8k['train'][0]['answer']

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

In [27]:
gsm8k_test = gsm8k['test']

In [28]:
prompt_complex = open('./chain-of-thought-hub/gsm8k/lib_prompt/prompt_hardest.txt').read()

In [29]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [30]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
)

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [31]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)

    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [32]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [33]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [43]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "Follow the given examples and answer the question. please answer the question step by step"},
        {"role": "user", "content": prompt_q}]
    )

In [44]:
response.choices[0].message.content

"Let's think step by step:\n\n1. Janet's ducks lay 16 eggs per day.\n2. She eats three eggs for breakfast.\n   Eggs remaining after breakfast = 16 - 3 = 13 eggs\n3. She uses four eggs for baking muffins for her friends.\n   Eggs remaining after baking = 13 - 4 = 9 eggs\n4. She sells the remaining 9 eggs at the farmers' market for $2 each.\n   Total earnings = 9 eggs * $2 = $18\n\nTherefore, Janet makes $18 every day at the farmers' market selling fresh duck eggs."

In [36]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(device_map="cpu")
compressed_prompt = llm_lingua.compress_prompt(prompt_q, target_token=300)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
compressed_prompt['compressed_prompt']

": Angelo andanie to plan many hours study their24 to They should study study total week they1 hour,3 breaks and\ns think step\nanie eachters hours hours\nate1 hours each hours\n andanie to start with planning,\n to. they203ute\nunch for  minutes hour3\nSo andanie study study,5 days to for\nThe4\n\n: Marks team52.  score teams\ns,5\n team also63,8\n and point\n0\ns scoreds scored4 half57The is The\n: B has two as than deck she2 more what of have she\n\n have5ella manybles as0\n Bella25 will\n more deck, have\n2+0\nThe\n4anas first2 many there\n\n\n,8\nets3anges theThe3 basket4\nThe1\n:13ruits between cost\n\n, then62\n1If\n of is and price\n can)2\n\n:y a large,0,, many the most\n\n,0y0 total\n,:0 Heanged into\n0 boxesters0He5 boxes0ese three to3, 1000 - 480 = 520 do not like to play basketball.\nThe percentage of the school that do not like to play basketball is 520/1000 * 100 = 52\nThe answer is 52\n\nQuestion: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every mor

In [48]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": compressed_prompt['compressed_prompt']},
    ]
)
ans_model = response.choices[0].message.content

In [49]:
ans_model

"Janet's ducks lay 16 eggs per day. She eats 3 for breakfast and bakes muffins for her friends with 4 eggs, which means she uses a total of 3 + 4 = 7 eggs for her personal use.\n\nSo, the remainder of eggs she can sell at the farmers' market is 16 - 7 = 9 eggs.\n\nIf Janet sells each fresh duck egg for $2, then she makes 9 * $2 = $18 every day at the farmers' market.\n\nTherefore, Janet makes $18 in dollars every day at the farmers' market."